# 🚀 VTrackAI Studio - GPU Backend (SAM3)

**Connect local frontend to Colab GPU for 10x faster SAM3 tracking!**

**Requirements:**
- Colab GPU runtime (T4/A100)
- Hugging Face account (free)

**Steps:**
1. Runtime → Change runtime type → T4 GPU
2. Run all cells (Ctrl+F9)
3. Copy **Cloudflare URL** (https://xxx.trycloudflare.com)
4. Local: `VITE_API_URL=<url> npm run dev`

## 1. Check GPU

In [ ]:
import torch
print(f"🔥 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU - Runtime > Change runtime type > GPU")

## 2. Clean Install + Dependencies

In [ ]:
!rm -rf /content/sam3 /content/vtrack-ai-studio
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q fastapi uvicorn[standard] python-multipart opencv-python librosa huggingface_hub
!pip install -q cloudflared nest-asyncio
print("✅ All dependencies ready")

## 3. Install SAM3 + VTrackAI

In [ ]:
!git clone https://github.com/facebookresearch/sam3
!pip install -e ./sam3 --quiet
!git clone https://github.com/Amitro123/vtrack-ai-studio
%cd vtrack-ai-studio/backend
!pip install -r requirements.txt --quiet
print("✅ SAM3 + VTrackAI installed")

## 4. Hugging Face Login + SAM3 Checkpoint

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import hf_hub_download
import os

os.makedirs("checkpoints/sam3", exist_ok=True)
checkpoint_path = hf_hub_download(
    repo_id="facebook/sam3-hiera-large",
    filename="sam3_hiera_large.pt",
    local_dir="./checkpoints/sam3"
)
print(f"✅ Checkpoint: {checkpoint_path}")

## 5. 🚀 Launch GPU Backend (Cloudflare Tunnel)

In [ ]:
import nest_asyncio
import uvicorn
import subprocess
import threading
import time
import requests

nest_asyncio.apply()

# Start FastAPI backend
def start_backend():
    uvicorn.run("server:app", host="0.0.0.0", port=8000, log_level="info")

backend_thread = threading.Thread(target=start_backend, daemon=True)
backend_thread.start()
time.sleep(8)  # Wait for startup

print("🚀 Backend started on http://localhost:8000")
print("⏳ Starting Cloudflare tunnel...")

# Cloudflare tunnel (FREE + stable)
!nohup cloudflared tunnel --url http://localhost:8000 > cloudflared.log 2>&1 &
time.sleep(8)

# Get Cloudflare URL from logs
!grep -o 'https://[^ ]*\.trycloudflare\.com' cloudflared.log || echo "🔍 Check cloudflared.log for URL"

print("\n" + "="*60)
print("🎉 VTrackAI GPU Backend LIVE!")
print("📡 Look for Cloudflare URL above (https://xxx.trycloudflare.com)")
print("="*60)
print("👉 Local frontend: VITE_API_URL=<cloudflare-url> npm run dev")

# Keep alive
try:
    while True:
        time.sleep(60)
except KeyboardInterrupt:
    print("\n👋 Backend stopped")

## 🔍 Health Check (Run after backend starts)

In [ ]:
# Test backend health
import requests
r = requests.get("http://localhost:8000/api/health")
print("Health:", r.json())
print("✅ GPU tracking ready! 🚀")